# 📊 TradingView ML Data Collector - Google Colab

**Complete machine learning data collection from TradingView**

This notebook provides:
- ✅ Automated market data collection
- ✅ Pre-built feature engineering
- ✅ ML-ready data preprocessing
- ✅ Example model training
- ✅ Real-time data access (with authentication)

---

## 🚀 Quick Start

1. Run **Section 1** to install dependencies
2. Run **Section 2** to set up the repository
3. Jump to any example you want to try!

---

## 📦 Section 1: Install Dependencies

Run this cell first to install all required packages.

In [ ]:
%%capture
# Install dependencies (silent installation)
!pip install pandas pyarrow scikit-learn numpy requests matplotlib seaborn

print("✅ Dependencies installed successfully!")

## 🔧 Section 2: Setup Repository

Clone the repository and checkout the ML branch.

In [ ]:
import os

# Check if already cloned
if os.path.exists('TradingView-Screener'):
    print("📁 Repository already exists, pulling latest changes...")
    %cd TradingView-Screener
    !git pull origin claude/data-collection-tool-012ct4SD9dgooKzsu9q71uL2
else:
    print("📥 Cloning repository...")
    !git clone https://github.com/jam2205/TradingView-Screener.git
    %cd TradingView-Screener
    !git checkout claude/data-collection-tool-012ct4SD9dgooKzsu9q71uL2

print("\n✅ Repository setup complete!")
print(f"📍 Current directory: {os.getcwd()}")

## 🎯 Section 3: Import Libraries

Import all the tools we need.

In [ ]:
# Core imports
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# TradingView Screener
from tradingview_screener import Query, col
from tradingview_screener.ml_collector import MLDataCollector
from tradingview_screener.features import (
    preprocess_for_ml,
    add_returns,
    add_price_momentum,
    add_volume_features,
    add_volatility,
    add_technical_flags,
    create_target_variable
)

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')

print("✅ All libraries imported successfully!")

## 🔐 Section 4: Authentication (Optional)

**For real-time data**, set your TradingView session cookie here.

**How to get your cookie:**
1. Go to [TradingView.com](https://www.tradingview.com) (logged in)
2. Press F12 → Application → Cookies → https://www.tradingview.com
3. Copy the `sessionid` value
4. Paste below

**Skip this section to use delayed data (15-min delay)**

In [ ]:
# Option 1: Paste your sessionid here
SESSIONID = ""  # Leave empty for delayed data

# Set up cookies
if SESSIONID:
    cookies = {'sessionid': SESSIONID}
    print("✅ Authentication enabled - Real-time data access")
else:
    cookies = None
    print("⚠️  No authentication - Using delayed data (15-min delay)")
    print("💡 Add your sessionid above for real-time data")

---

# 📊 EXAMPLES

---

## 🎯 Example 1: Simple Data Collection

Collect basic market data for large-cap stocks.

In [ ]:
# Initialize data collector
collector = MLDataCollector(
    output_dir='/content/data',
    format='parquet',
    add_collection_metadata=True
)

# Define query
query = (
    Query()
    .select('name', 'close', 'volume', 'market_cap_basic', 'RSI', 'MACD.macd', 'change')
    .where(
        col('market_cap_basic') > 10_000_000_000,  # > $10B market cap
        col('volume') > 1_000_000,  # > 1M volume
    )
    .order_by('volume', ascending=False)
    .limit(100)
)

# Collect data
print("🔍 Scanning markets...\n")
df = collector.collect_once(query, dataset_name='large_caps', cookies=cookies)

print(f"\n✅ Collected {len(df)} stocks")
print(f"📊 Columns: {list(df.columns)}")

# Display results
df.head(10)

In [ ]:
# Quick statistics
print("📈 Summary Statistics:\n")
print(f"Average Price: ${df['close'].mean():.2f}")
print(f"Total Volume: {df['volume'].sum():,.0f}")
print(f"Average RSI: {df['RSI'].mean():.2f}")
print(f"Average Change: {df['change'].mean():.2f}%")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Volume distribution
df['volume'].hist(bins=30, ax=axes[0], color='skyblue', edgecolor='black')
axes[0].set_title('Volume Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Volume')
axes[0].set_ylabel('Frequency')

# RSI distribution
df['RSI'].hist(bins=30, ax=axes[1], color='lightcoral', edgecolor='black')
axes[1].set_title('RSI Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('RSI')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 🎯 Example 2: Feature Engineering

Add technical features to the collected data.

In [ ]:
# Collect data with more fields
query = (
    Query()
    .select(
        'name', 'close', 'open', 'high', 'low', 'volume',
        'market_cap_basic', 'relative_volume_10d_calc',
        'RSI', 'MACD.macd', 'MACD.signal',
        'EMA5', 'EMA20', 'EMA50'
    )
    .where(
        col('market_cap_basic') > 5_000_000_000,
        col('volume') > 500_000
    )
    .limit(200)
)

df = collector.collect_once(query, dataset_name='feature_engineering', cookies=cookies)
print(f"✅ Collected {len(df)} stocks\n")

# Add custom features
print("🔧 Engineering features...\n")

# Add returns
df = add_returns(df, price_column='close', periods=[1, 5, 10])
print("  ✓ Added returns")

# Add momentum
df = add_price_momentum(df, price_column='close', windows=[5, 10, 20])
print("  ✓ Added momentum")

# Add volume features
df = add_volume_features(df, volume_column='volume', windows=[5, 10])
print("  ✓ Added volume features")

# Add volatility
df = add_volatility(df, price_column='close', windows=[5, 10, 20])
print("  ✓ Added volatility")

# Add technical flags
df = add_technical_flags(df, price_column='close', volume_column='volume')
print("  ✓ Added technical flags")

print(f"\n✅ Total features: {len(df.columns)}")
print(f"\n📊 New columns added:")
new_cols = [c for c in df.columns if any(x in c for x in ['return', 'momentum', 'volatility', 'above', 'bullish'])]
for col in new_cols[:15]:
    print(f"  • {col}")
if len(new_cols) > 15:
    print(f"  ... and {len(new_cols) - 15} more")

## 🎯 Example 3: Complete ML Pipeline

Collect data, preprocess, and train a model to predict price direction.

In [ ]:
# Step 1: Collect comprehensive data
print("📊 Step 1: Collecting market data...\n")

query = (
    Query()
    .select(
        'name', 'close', 'open', 'high', 'low', 'volume',
        'market_cap_basic', 'relative_volume_10d_calc',
        'RSI', 'MACD.macd', 'MACD.signal',
        'EMA5', 'EMA20', 'EMA50', 'EMA200',
        'change'
    )
    .where(
        col('market_cap_basic') > 5_000_000_000,
        col('volume') > 500_000
    )
    .order_by('volume', ascending=False)
    .limit(500)
)

df = collector.collect_once(query, dataset_name='ml_training', cookies=cookies)
print(f"✅ Collected {len(df)} stocks\n")

# Step 2: Preprocess for ML
print("🔧 Step 2: Preprocessing for ML...\n")

df_ml = preprocess_for_ml(
    df,
    price_column='close',
    volume_column='volume',
    target_type='direction',  # Binary: predict up/down
    target_periods=1,  # Predict 1 period ahead
    add_technical=True,
    normalize=True,
    handle_missing='median',
    remove_outlier=True
)

print(f"✅ Preprocessed data shape: {df_ml.shape}")
print(f"✅ Features created: {df_ml.shape[1]} columns\n")

# Check target distribution
target_dist = df_ml['target'].value_counts()
print("📊 Target distribution:")
print(f"  Down (0): {target_dist.get(0, 0)} ({target_dist.get(0, 0)/len(df_ml)*100:.1f}%)")
print(f"  Up (1):   {target_dist.get(1, 0)} ({target_dist.get(1, 0)/len(df_ml)*100:.1f}%)")

In [ ]:
# Step 3: Prepare data for training
print("🎯 Step 3: Preparing train/test split...\n")

# Select features
exclude_cols = ['ticker', 'name', 'dataset_name', 'collection_timestamp', 'collection_unix', 'target']
feature_cols = [c for c in df_ml.columns if c not in exclude_cols]

# Get numeric features only
X = df_ml[feature_cols].select_dtypes(include=['number'])
y = df_ml['target']

# Remove any remaining NaN
valid_idx = ~(X.isnull().any(axis=1) | y.isnull())
X = X[valid_idx]
y = y[valid_idx]

print(f"✅ Final dataset: {len(X)} samples, {len(X.columns)} features")

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"✅ Training set: {len(X_train)} samples")
print(f"✅ Test set: {len(X_test)} samples\n")

# Step 4: Train model
print("🤖 Step 4: Training Random Forest model...\n")

model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=10,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)
print("✅ Model trained!\n")

# Step 5: Evaluate
print("📊 Step 5: Model Evaluation\n")
print("=" * 50)

# Predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Accuracy
train_acc = accuracy_score(y_train, y_pred_train)
test_acc = accuracy_score(y_test, y_pred_test)

print(f"\n🎯 Accuracy Scores:")
print(f"  Training:   {train_acc:.2%}")
print(f"  Test:       {test_acc:.2%}")
print(f"  Difference: {abs(train_acc - test_acc):.2%}")

# Classification report
print(f"\n📋 Classification Report (Test Set):\n")
print(classification_report(y_test, y_pred_test, target_names=['Down', 'Up']))

# Confusion matrix
print(f"\n🔢 Confusion Matrix:\n")
cm = confusion_matrix(y_test, y_pred_test)
print(cm)
print(f"\n(Rows: Actual, Columns: Predicted)")

In [ ]:
# Feature importance
print("🔍 Top 15 Most Important Features:\n")

feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print(feature_importance.head(15).to_string(index=False))

# Visualize top features
plt.figure(figsize=(12, 6))
top_features = feature_importance.head(15)
plt.barh(range(len(top_features)), top_features['importance'], color='steelblue')
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Importance', fontsize=12)
plt.title('Top 15 Feature Importances', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 🎯 Example 4: Preset Scans

Try different preset scanning strategies.

In [ ]:
# Strategy 1: Large Cap Momentum
print("🚀 Large Cap Momentum Scan\n")

query = (
    Query()
    .select('name', 'close', 'volume', 'market_cap_basic', 'RSI', 'MACD.macd', 'change', 'Perf.W')
    .where(
        col('market_cap_basic') > 10_000_000_000,
        col('volume') > 1_000_000,
        col('relative_volume_10d_calc') > 1.2
    )
    .order_by('volume', ascending=False)
    .limit(50)
)

df_momentum = collector.collect_once(query, dataset_name='momentum', cookies=cookies)
print(f"✅ Found {len(df_momentum)} momentum stocks\n")
df_momentum.head(10)

In [ ]:
# Strategy 2: Oversold Stocks (RSI < 30)
print("❄️ Oversold Stocks Scan\n")

query = (
    Query()
    .select('name', 'close', 'volume', 'RSI', 'change', 'market_cap_basic')
    .where(
        col('RSI') < 30,
        col('volume') > 500_000,
        col('market_cap_basic') > 1_000_000_000
    )
    .order_by('RSI', ascending=True)
    .limit(50)
)

df_oversold = collector.collect_once(query, dataset_name='oversold', cookies=cookies)
print(f"✅ Found {len(df_oversold)} oversold stocks\n")
df_oversold.head(10)

In [ ]:
# Strategy 3: Crypto Momentum
print("🪙 Crypto Momentum Scan\n")

query = (
    Query()
    .set_markets('crypto')
    .select('name', 'close', 'volume', 'market_cap_calc', 'RSI', 'Perf.W', 'change')
    .where(
        col('market_cap_calc') > 100_000_000,
        col('relative_volume_10d_calc') > 1.5
    )
    .order_by('volume', ascending=False)
    .limit(50)
)

df_crypto = collector.collect_once(query, dataset_name='crypto', cookies=cookies)
print(f"✅ Found {len(df_crypto)} crypto assets\n")
df_crypto.head(10)

## 🎯 Example 5: Export Data

Save your collected data in different formats.

In [ ]:
# Export to different formats
print("💾 Exporting data...\n")

# Create export directory
!mkdir -p /content/exports

# CSV
df.to_csv('/content/exports/market_data.csv', index=False)
print("✅ Saved: /content/exports/market_data.csv")

# Parquet (best for ML)
df.to_parquet('/content/exports/market_data.parquet', index=False)
print("✅ Saved: /content/exports/market_data.parquet")

# JSON
df.to_json('/content/exports/market_data.json', orient='records', indent=2)
print("✅ Saved: /content/exports/market_data.json")

print("\n📥 Files ready for download from /content/exports/")

# Show file sizes
!ls -lh /content/exports/

---

## 🎨 Custom Query Builder

Build your own custom queries here!

In [ ]:
# Build your custom query
custom_query = (
    Query()
    .select(
        'name',
        'close',
        'volume',
        # Add more columns here...
    )
    .where(
        # Add your filters here...
        col('volume') > 100_000,
    )
    .order_by('volume', ascending=False)
    .limit(100)
)

# Execute
df_custom = collector.collect_once(custom_query, dataset_name='custom', cookies=cookies)
print(f"✅ Found {len(df_custom)} results")
df_custom.head()

---

## 💡 Pro Tips

### Available Fields

**Price & Volume:**
- `close`, `open`, `high`, `low`
- `volume`, `relative_volume_10d_calc`
- `change`

**Technical Indicators:**
- `RSI`
- `MACD.macd`, `MACD.signal`
- `EMA5`, `EMA20`, `EMA50`, `EMA200`
- `VWAP`
- `volatility.D`

**Fundamentals:**
- `market_cap_basic`
- `price_earnings_ttm`
- `dividend_yield_recent`
- `earnings_per_share_basic_ttm`

**Performance:**
- `Perf.W` (1 week)
- `Perf.1M` (1 month)
- `Perf.3M` (3 months)

[See full field list →](https://shner-elmo.github.io/TradingView-Screener/fields/stocks.html)

### Markets Available
- `america` (US stocks)
- `crypto`
- `forex`
- `cfd`
- `futures`
- `bonds`

### Filter Operations
```python
col('price') > 100
col('price').between(10, 50)
col('volume').isin([1000, 2000, 3000])
col('RSI') < 30
col('MACD.macd') >= col('MACD.signal')
```

---

## 📚 Resources

- **Documentation:** [ML_GUIDE.md](https://github.com/jam2205/TradingView-Screener/blob/claude/data-collection-tool-012ct4SD9dgooKzsu9q71uL2/ML_GUIDE.md)
- **Quick Start:** [ML_QUICKSTART.md](https://github.com/jam2205/TradingView-Screener/blob/claude/data-collection-tool-012ct4SD9dgooKzsu9q71uL2/ML_QUICKSTART.md)
- **Live Scanner Guide:** [LIVE_SCANNER_GUIDE.md](https://github.com/jam2205/TradingView-Screener/blob/claude/data-collection-tool-012ct4SD9dgooKzsu9q71uL2/LIVE_SCANNER_GUIDE.md)
- **Examples:** [examples/](https://github.com/jam2205/TradingView-Screener/tree/claude/data-collection-tool-012ct4SD9dgooKzsu9q71uL2/examples)

---

## 🎉 You're Ready!

Start collecting data and building your ML models!

**Happy Trading! 📊🚀**